In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [ ]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    if city not in cities:
        cities.append(city)

print(f"Number of cities in the list: {len(cities)}")

In [ ]:
# Set the API base URL
url = "http://api.openweathermap.org/data/2.5/weather"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

record_count = 1
set_count = 1

# Loop the cities
for i, city in enumerate(cities):
        
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    city_url = f"{url}?q={city}&appid={weather_api_key}"

    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()

        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
    
    except Exception as e:
        print(f"City not found. Skipping... {e}")
        pass

# Print the Data Retrieval Complete message
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


In [ ]:
# Convert city_data into a pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Display the DataFrame
print(city_data_df.head())

# Show Record Count
print(f"Record Count: {len(city_data_df)}")


In [ ]:
city_data_df.head()


In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

In [ ]:
# Build scatter plot for latitude vs. temperature
plt.figure(figsize=(8, 6))
plt.scatter(city_data_df["Lat"], city_data_df["Max Temp"], edgecolors="black", alpha=0.7)

plt.xlabel("Latitude")
plt.ylabel("Max Temperature (°F)")
plt.title("City Latitude vs. Max Temperature")
plt.grid(True)
plt.tight_layout()

plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()


In [ ]:
# Build the scatter plot for latitude vs. humidity
plt.figure(figsize=(8, 6))
plt.scatter(city_data_df["Lat"], city_data_df["Humidity"], edgecolors="black", alpha=0.7)

# Incorporate the other graph properties
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.title("City Latitude vs. Humidity")
plt.grid(True)
plt.tight_layout()

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()


In [ ]:
# Build the scatter plot for latitude vs. cloudiness
plt.figure(figsize=(8, 6))
plt.scatter(city_data_df["Lat"], city_data_df["Cloudiness"], edgecolors="black", alpha=0.7)

# Incorporate the other graph properties
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.title("City Latitude vs. Cloudiness")
plt.grid(True)
plt.tight_layout()

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()


In [ ]:
# Build the scatter plot for latitude vs. wind speed
plt.figure(figsize=(8, 6))
plt.scatter(city_data_df["Lat"], city_data_df["Wind Speed"], edgecolors="black", alpha=0.7)

# Incorporate the other graph properties
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.title("City Latitude vs. Wind Speed")
plt.grid(True)
plt.tight_layout()

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()


In [ ]:
# Define a function to create Linear Regression plots
def plot_linear_regression(x_values, y_values, title, xlabel, ylabel, text_coordinates):
    # Perform linear regression on x and y values
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))

    # Create scatter plot
    plt.scatter(x_values, y_values)
    plt.plot(x_values, regress_values, "r-")

    # Annotate the linear regression
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")

    # Label plot
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    # Print r-value
    print(f"The r-value is: {rvalue}")

    # Show plot
    plt.show()

# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df[city_data_df["Lat"] >= 0]

# Display sample data
print(northern_hemi_df.head())


In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df[city_data_df["Lat"] < 0]

# Display sample data
print(southern_hemi_df.head())


In [ ]:
# Temperature vs. Latitude Linear Regression Plot
# Linear regression on Northern Hemisphere
x_values_north = northern_hemi_df["Lat"]
y_values_north = northern_hemi_df["Max Temp"]

plot_linear_regression(x_values_north, y_values_north, 
                       "Linear Regression: Temperature vs. Latitude (Northern Hemisphere)", 
                       "Latitude", "Max Temperature (°F)", (0, 30))


In [ ]:
# Temperature vs. Latitude Linear Regression Plot
# Linear regression on Southern Hemisphere
x_values_south = southern_hemi_df["Lat"]
y_values_south = southern_hemi_df["Max Temp"]

plot_linear_regression(x_values_south, y_values_south, 
                       "Linear Regression: Temperature vs. Latitude (Southern Hemisphere)", 
                       "Latitude", "Max Temperature (°F)", (-50, 80))

# Discussion about the linear relationship:
# The Northern Hemisphere have a negative correlation and is much more condense compaired to the Southern Hemisphere which is more scattered and have a Positive correlation.  

In [ ]:
# Humidity vs. Latitude Linear Regression Plot
# Linear regression on Northern Hemisphere for Humidity vs. Latitude
x_values_north_humidity = northern_hemi_df["Lat"]
y_values_north_humidity = northern_hemi_df["Humidity"]

plot_linear_regression(x_values_north_humidity, y_values_north_humidity, 
                       "Linear Regression: Humidity vs. Latitude (Northern Hemisphere)", 
                       "Latitude", "Humidity (%)", (40, 10))


In [ ]:
# Humidity vs. Latitude Linear Regression Plot
# Linear regression on Southern Hemisphere for Humidity vs. Latitude
x_values_south_humidity = southern_hemi_df["Lat"]
y_values_south_humidity = southern_hemi_df["Humidity"]

plot_linear_regression(x_values_south_humidity, y_values_south_humidity, 
                       "Linear Regression: Humidity vs. Latitude (Southern Hemisphere)", 
                       "Latitude", "Humidity (%)", (-50, 15))

# Discussion about the linear relationship:
# The Northern Hemisphere have a positive correlation and the Southern Hemisphere have a pretty even corerlation and both are very scattered. .  

In [ ]:
# Cloudiness vs. Latitude Linear Regression Plot
# Linear regression on Northern Hemisphere for Cloudiness vs. Latitude
x_values_north_cloudiness = northern_hemi_df["Lat"]
y_values_north_cloudiness = northern_hemi_df["Cloudiness"]

plot_linear_regression(x_values_north_cloudiness, y_values_north_cloudiness, 
                       "Linear Regression: Cloudiness vs. Latitude (Northern Hemisphere)", 
                       "Latitude", "Cloudiness (%)", (40, 50))


In [ ]:
# Cloudiness vs. Latitude Linear Regression Plot
# Linear regression on Southern Hemisphere for Cloudiness vs. Latitude
x_values_south_cloudiness = southern_hemi_df["Lat"]
y_values_south_cloudiness = southern_hemi_df["Cloudiness"]

plot_linear_regression(x_values_south_cloudiness, y_values_south_cloudiness, 
                       "Linear Regression: Cloudiness vs. Latitude (Southern Hemisphere)", 
                       "Latitude", "Cloudiness (%)", (-50, 50))

# Discussion about the linear relationship:
# Both Northern Hemisphere and the Southern Hemisphere have a positive correlation, Northern hemisphere have more range and more consistency of 100% clouds and 0% cloudswhire the southern hemisphere have only a consistency of 100% clouds. 

In [ ]:
# Wind Speed vs. Latitude Linear Regression Plot
# Linear regression on Northern Hemisphere for Wind Speed vs. Latitude
x_values_north_wind_speed = northern_hemi_df["Lat"]
y_values_north_wind_speed = northern_hemi_df["Wind Speed"]

plot_linear_regression(x_values_north_wind_speed, y_values_north_wind_speed, 
                       "Linear Regression: Wind Speed vs. Latitude (Northern Hemisphere)", 
                       "Latitude", "Wind Speed (mph)", (40, 25))


In [ ]:
# Wind Speed vs. Latitude Linear Regression Plot
# Linear regression on Southern Hemisphere for Wind Speed vs. Latitude
x_values_south_wind_speed = southern_hemi_df["Lat"]
y_values_south_wind_speed = southern_hemi_df["Wind Speed"]

plot_linear_regression(x_values_south_wind_speed, y_values_south_wind_speed, 
                       "Linear Regression: Wind Speed vs. Latitude (Southern Hemisphere)", 
                       "Latitude", "Wind Speed (mph)", (-50, 25))

# Discussion about the linear relationship:
# The Northern Hemisphere have a positive correlation and more condensed compaited to the Southern Hemisphere which have a negative correlation and are more scattered. .  